In [1]:
import pandas as pd
import numpy as np
import preprocess_text as pt

In [12]:
train_df = pd.read_csv('data/train.csv')

In [3]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
print(df.values[0])

[0 1 2 'What is the step by step guide to invest in share market in india?'
 'What is the step by step guide to invest in share market?' 0]


In [5]:
pre = pt.Preprocess_text(df, columns=['question1', 'question2'], word2vec_path="data/GoogleNews-vectors-negative300.bin")

[nltk_data] Downloading package punkt to /Users/joarisha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
w2v model generated.


In [7]:
all_word_vecs = pre.tokenize_data()

Tokenizing question1
Tokenizing question2


In [14]:
np.save("tokenized_words.npy", all_word_vecs)

In [8]:
del pre

In [9]:
print(all_word_vecs[0][5])

[61027, 0, 4502, 249, 0, 0, 4129, 2823, 6027, 0, 2823, 1546, 0, 84, 184, 4, 160, 47, 171, 0]


In [9]:
i2w = pre.index2word

In [10]:
print([i2w[idx] for idx in all_word_vecs[0][1]])

['what', 'is', 'the', 'story', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'diamond', 'UNK']


In [11]:
df.question1.values[1]

'What is the story of Kohinoor (Koh-i-Noor) Diamond?'

In [15]:
#df2 = pd.DataFrame()
#df2 = pre.idx2vector.values()
#del pre
#num_w2v_words = 3000001
#vectors = np.array([val for arrval in df2 for val in arrval]).reshape((num_w2v_words, 300))

In [25]:
np.savez("word2vec_mine.npz",vectors = pre.vectors, words=[])

In [2]:
#embedding_data = np.load("word2vec_mine.npz")
embedding_data = np.load("data/weights.npy")

It is quite obvious that better tokenizing and pre-processing would help.

In [3]:
import CNNSentence

Using TensorFlow backend.


In [4]:
cnns = CNNSentence.CNNSentence(num_inputs=2, max_doc_length=15, output_name_size={"classes": 2}, embedding_vectors=embedding_data)

/Users/joarisha/personalProjects/QuoraQuestionPairs/CNNSentence.py:23: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return merge(kernels, mode='concat', concat_axis=2, name=kernel_name)
/Users/joarisha/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [6]:
cnns.model.get_layer("embeddings").trainable = False

In [25]:
batch_size = 100
train_val_split = 0.8

In [7]:
all_word_vecs = np.load("tokenized_words.npy")

In [13]:
y = train_df.is_duplicate.values

In [14]:
tind = np.random.rand(len(train_df)) < train_val_split

In [15]:
x1_pad = CNNSentence.get_padded_input(all_word_vecs[0])

In [16]:
x2_pad = CNNSentence.get_padded_input(all_word_vecs[1])

In [23]:
y_oh = CNNSentence.one_hot_words(y)

In [24]:
x1_train = x1_pad[tind,:]
x2_train = x2_pad[tind,:]
x1_val = x1_pad[~tind,:]
x2_val = x2_pad[~tind,:]
y_train = y_oh[tind,:]
y_val = y_oh[~tind,:]

In [27]:
model = cnns.model

In [29]:
import keras

In [30]:
optim = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=optim,
              metrics=['accuracy'])

history = model.fit([x1_train, x2_train], y_train,
                    batch_size=batch_size,
                    epochs=1,
                    verbose=1,
                    validation_split=0.1)

Train on 291006 samples, validate on 32335 samples
Epoch 1/1
291006/291006 [==============================] - 251s - loss: 0.5465 - acc: 0.7282 - val_loss: 0.5106 - val_acc: 0.7548

NameError: name 'x_test' is not defined

In [31]:
score = model.evaluate([x1_val, x2_val], y_val,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

80900/80949 [============================>.] - ETA: 0s 